In [2]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr

ModuleNotFoundError: No module named 'pydub'

In [10]:
def convert_mp3_to_wav(mp3_path, wav_path):
    """Convert MP3 file to WAV format."""
    try:
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        print(f"Converted {mp3_path} to {wav_path}")
    except Exception as e:
        print(f"Error converting {mp3_path} to WAV: {e}")

def transcribe_audio_with_timestamps(audio_path, output_text_path, interval=30):
    """Transcribe WAV audio file to text with timestamps and save it."""
    recognizer = sr.Recognizer()
    
    try:
        audio = AudioSegment.from_wav(audio_path)
        duration_ms = len(audio)  # Total duration in milliseconds
        interval_ms = interval * 1000  # Convert interval to milliseconds
        
        current_time_ms = 0
        
        with open(output_text_path, 'w', encoding='utf-8') as f:
            while current_time_ms < duration_ms:
                end_time_ms = min(current_time_ms + interval_ms, duration_ms)
                segment = audio[current_time_ms:end_time_ms]
                
                # Save the segment as a temporary file for recognition
                temp_file = 'temp_segment.wav'
                segment.export(temp_file, format='wav')
                
                with sr.AudioFile(temp_file) as source:
                    audio_data = recognizer.record(source)
                    
                    try:
                        text = recognizer.recognize_google(audio_data, language="ko-KR")
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] {text}\n")
                    except sr.UnknownValueError:
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] 음성을 인식할 수 없습니다.\n")
                    except sr.RequestError as e:
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] 요청 실패: {e}\n")
                
                current_time_ms = end_time_ms
            
        os.remove(temp_file)  # Clean up the temporary file
        print(f"Transcription completed for {audio_path}")
    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")

# Path to the folder containing the audio files
folder_path = 'ebs_audio(2)'  

for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp3'):  # Assuming the audio files are in .mp3 format
        mp3_file_path = os.path.join(folder_path, file_name)
        wav_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}.wav")
        output_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_transcription.txt")
        
        # Convert MP3 to WAV
        convert_mp3_to_wav(mp3_file_path, wav_file_path)
        
        # Transcribe audio with timestamps
        transcribe_audio_with_timestamps(wav_file_path, output_file_path)
        
        # Optionally remove the WAV file after transcription
        os.remove(wav_file_path)
        
        print(f"Process completed for {file_name}")

Converted ebs_audio_combined_1~45\01강_combined.mp3 to ebs_audio_combined_1~45\01강_combined.wav
Transcription completed for ebs_audio_combined_1~45\01강_combined.wav
Process completed for 01강_combined.mp3
Converted ebs_audio_combined_1~45\02강_combined.mp3 to ebs_audio_combined_1~45\02강_combined.wav
Transcription completed for ebs_audio_combined_1~45\02강_combined.wav
Process completed for 02강_combined.mp3
Converted ebs_audio_combined_1~45\03강_combined.mp3 to ebs_audio_combined_1~45\03강_combined.wav
Transcription completed for ebs_audio_combined_1~45\03강_combined.wav
Process completed for 03강_combined.mp3
Converted ebs_audio_combined_1~45\04강_combined.mp3 to ebs_audio_combined_1~45\04강_combined.wav
Transcription completed for ebs_audio_combined_1~45\04강_combined.wav
Process completed for 04강_combined.mp3
Converted ebs_audio_combined_1~45\05강_combined.mp3 to ebs_audio_combined_1~45\05강_combined.wav
Transcription completed for ebs_audio_combined_1~45\05강_combined.wav
Process completed for 05

In [ ]:
import os
from pydub import AudioSegment
import speech_recognition as sr
from concurrent.futures import ThreadPoolExecutor

def convert_mp3_to_wav(mp3_path, wav_path):
    """Convert MP3 file to WAV format."""
    try:
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        print(f"Converted {mp3_path} to {wav_path}")
    except Exception as e:
        print(f"Error converting {mp3_path} to WAV: {e}")

def transcribe_audio_with_timestamps(audio_path, output_text_path, interval=30):
    """Transcribe WAV audio file to text with timestamps and save it."""
    recognizer = sr.Recognizer()
    
    try:
        audio = AudioSegment.from_wav(audio_path)
        duration_ms = len(audio)  # Total duration in milliseconds
        interval_ms = interval * 1000  # Convert interval to milliseconds
        
        current_time_ms = 0
        
        with open(output_text_path, 'w', encoding='utf-8') as f:
            while current_time_ms < duration_ms:
                end_time_ms = min(current_time_ms + interval_ms, duration_ms)
                segment = audio[current_time_ms:end_time_ms]
                
                # Use in-memory bytes instead of a temporary file
                segment_bytes = segment.export(format='wav').read()
                
                with sr.AudioFile(segment_bytes) as source:
                    audio_data = recognizer.record(source)
                    
                    try:
                        text = recognizer.recognize_google(audio_data, language="ko-KR")
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] {text}\n")
                    except sr.UnknownValueError:
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] 음성을 인식할 수 없습니다.\n")
                    except sr.RequestError as e:
                        f.write(f"[{int(current_time_ms / 1000)} - {int(end_time_ms / 1000)}] 요청 실패: {e}\n")
                
                current_time_ms = end_time_ms
            
        print(f"Transcription completed for {audio_path}")
    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")

def process_file(file_name, folder_path):
    if file_name.endswith('.mp3'):  # Assuming the audio files are in .mp3 format
        mp3_file_path = os.path.join(folder_path, file_name)
        wav_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}.wav")
        output_file_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}_transcription.txt")
        
        # Convert MP3 to WAV
        convert_mp3_to_wav(mp3_file_path, wav_file_path)
        
        # Transcribe audio with timestamps
        transcribe_audio_with_timestamps(wav_file_path, output_file_path)
        
        # Optionally remove the WAV file after transcription
        os.remove(wav_file_path)
        
        print(f"Process completed for {file_name}")

# Path to the folder containing the audio files
folder_path = 'ebs_audio(2)'  

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    for file_name in os.listdir(folder_path):
        executor.submit(process_file, file_name, folder_path)